In [2]:
import altair as alt
import numpy as np
import pandas as pd

from datetime import timedelta

In [39]:
chosen_owners = ["Eli", "Heather", "Kensie", "Sam", "Rachel"]
days = 140

In [40]:
# Load Data
df = pd.read_json("workitems.json")
df["iterationStartDate"] = pd.to_datetime(df["iterationStartDate"])
df["workStartDate"] = pd.to_datetime(df["workStartDate"])
df["finishDate"] = pd.to_datetime(df["finishDate"])

In [46]:
# Filter Data based on `chosen_owners` and `days`
filtered = df[df.owner.isin(chosen_owners)]

end_date = filtered["iterationStartDate"].max()
start_date = end_date - timedelta(days)

filtered = filtered[filtered.iterationStartDate.between(start_date, end_date)]

In [42]:
#df

In [47]:
# Wrangle data to get cumulative points over time
date_range = pd.date_range(start_date, end_date)
points_over_time = pd.pivot_table(filtered, values="points", index=["finishDate"], columns=["owner"], aggfunc=np.sum, fill_value=0)
points_over_time = points_over_time.reindex(date_range, fill_value=0)
points_over_time = points_over_time.cumsum()
points_over_time = points_over_time.T.reset_index()
points_over_time = pd.melt(points_over_time, id_vars=["owner"], var_name="date", value_name="points")

In [44]:
#points_over_time

# Eli Newguy Performance Review

Judging by story points, I went from worst to first in 4 months!

In [48]:
alt.Chart(points_over_time).mark_line().encode(
    x = 'date',
    y = 'points',
    color = 'owner'
)

alt.Chart(...)

## Interactivity in Jupyter

Requires [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/index.html) to be installed into your Python environment and Jupyter instance

In [23]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

### Interactive Filtering of Data

In [21]:
@interact
def filter_df(owner=["Eli", "Heather", "Kensie", "Rachel", "Sam"], days=(14, 150, 7)):
    chosen_owners = [owner]
    filtered = df[df.owner.isin(chosen_owners)]

    end_date = filtered["iterationStartDate"].max()
    start_date = end_date - timedelta(days)
    return filtered[filtered.iterationStartDate.between(start_date, end_date)]

interactive(children=(Dropdown(description='owner', options=('Eli', 'Heather', 'Kensie', 'Rachel', 'Sam'), val…

### Interactive plot of story points over time

In [22]:
@interact
def filter_df(owner=["Eli", "Heather", "Kensie", "Rachel", "Sam"], days=(14, 150, 7)):
    chosen_owners = [owner]
    filtered = df[df.owner.isin(chosen_owners)]

    end_date = filtered["iterationStartDate"].max()
    start_date = end_date - timedelta(days)
    filtered = filtered[filtered.iterationStartDate.between(start_date, end_date)]
    
    # Wrangle data to get cumulative points over time
    date_range = pd.date_range(start_date, end_date)
    points_over_time = pd.pivot_table(filtered, values="points", index=["finishDate"], columns=["owner"], aggfunc=np.sum, fill_value=0)
    points_over_time = points_over_time.reindex(date_range, fill_value=0)
    points_over_time = points_over_time.cumsum()
    points_over_time = points_over_time.T.reset_index()
    points_over_time = pd.melt(points_over_time, id_vars=["owner"], var_name="date", value_name="points")
    
    return alt.Chart(points_over_time).mark_line().encode(
        x = 'date',
        y = 'points',
        color = 'owner'
    )


interactive(children=(Dropdown(description='owner', options=('Eli', 'Heather', 'Kensie', 'Rachel', 'Sam'), val…